# Weather and Motor Vehicle Collisions

In [1]:
import pandas as pd
import datetime
from datetime import date
from dateutil.rrule import rrule, DAILY
from __future__ import division

pd.set_option('display.max_columns', None)

## Download weather data

In [ ]:
start_date = date(2012, 7, 1)
end_date = date(2016, 2, 29)

# data = pd.DataFrame()
frames = []
url_template = 'https://www.wunderground.com/history/airport/KNYC/%s/%s/%s/DailyHistory.html?req_city=New+York&req_state=NY&req_statename=New+York&reqdb.zip=10001&reqdb.magic=4&reqdb.wmo=99999&format=1.csv'

month = ""

for dt in rrule(DAILY, dtstart=start_date, until=end_date):
    if (month != dt.strftime("%m")):
        month = dt.strftime("%m")
        print 'Downloading to memory: ' + dt.strftime("%Y-%m")    
    frames.append(pd.read_csv(url_template % (dt.strftime("%Y"),dt.strftime("%m"), dt.strftime("%d"))))

print "Saving data to csv..."
data = pd.concat(frames)
data.to_csv('weather_data_nyc.csv', sep=',')

## Cleaning the weather dataset
### Convert weather DateUTC to local time

In [2]:
from datetime import datetime
from dateutil import tz

weather = pd.read_csv('datasets/weather_data_nyc_clean.csv')

def UTCtoActual(utcDate):
    from_zone = tz.gettz('UTC')
    to_zone = tz.gettz('America/New_York')
    
    utc = datetime.strptime(utcDate.DateUTC, '%m/%d/%Y %H:%M:%S')\
                  .replace(tzinfo=from_zone)\
                  .astimezone(to_zone)
    s = pd.Series([utc.year, utc.month, utc.day, utc.hour])
    s.columns = ['Year', 'Month', 'Day', 'Hour']
    return s
    
#weather['DateActual'] = weather.DateUTC.map()

In [17]:
weather[['Year', 'Month', 'Day', 'Hour']] = weather.apply(UTCtoActual, axis=1)
weather.to_csv('datasets/weather_data_nyc_clean2.csv')

### Merge weather and NYPD MVC datasets

In [2]:
incidents = pd.read_csv('datasets/NYPD_Motor_Vehicle_Collisions.csv')
weather = pd.read_csv('datasets/weather_data_nyc_clean2.csv')
weather.head(1)

,Unnamed: 0,Conditions,DateUTC,Dew PointC,Events,Gust SpeedKm/h,Humidity,Precipitationmm,Sea Level PressurehPa,TemperatureC,TimeEDT,TimeEST,VisibilityKm,Wind Direction,Wind SpeedKm/h,WindDirDegrees,Year,Month,Day,Hour
0,0,Clear,7/1/2012 4:51:00,17.2,NaN,NaN,58.0,NaN,1008.6,26.1,12:51 AM,NaN,16.1,West,13,280,2012,7,1,0


In [ ]:
weather[(weather.Year == 2015) & (weather.Month == 11) & (weather.Day == 27)]

In [146]:
features0 = ['Conditions', 'TemperatureC']
features = ['Conditions', \
            'Precipitationmm', \
            'TemperatureC', 'VisibilityKm']

def lookup_weather2(year, month, day, hour):
    w = weather[(weather.Year == year) & (weather.Month == month) & (weather.Day == day) & (weather.Hour == hour)]
    return w

def lookup_weather(date, time):
    month = int(date.split('/')[0])
    day = int(date.split('/')[1])
    year = int(date.split('/')[2])
    hour = int(time.split(':')[0])
    d = lookup_weather2(year, month, day, hour).head(1)
    if (d.empty):
        dt_back = datetime.datetime(year, month, day, hour) - datetime.timedelta(hours=1)
        dt_forward = datetime.datetime(year, month, day, hour) + datetime.timedelta(hours=1)
        
        d_back = lookup_weather2(dt_back.year, dt_back.month, dt_back.day, dt_back.hour)
        if (not d_back.empty): return d_back
        
        d_forward = lookup_weather2(dt_forward.year, dt_forward.month, dt_forward.day, dt_forward.hour)
        if (not d_forward.empty): return d_forward
    return d



def merge_weather(incident):
    date = incident.DATE
    time = incident.TIME
    #print "0"
    w = lookup_weather(date, time)
    #[unnamed, condition, dateUTC, Dew, Events, Gust, Humidity,Precipitationmm,Sea_Level_PressurehPa, TemperatureC] = w.values[0]

    #print "1"
    try:
        #print "2"
        #print w
        con = "-"
        temp = "-"
        rainmm = "-"
        viskm = "-"
        #print "2.5"
        if (not pd.isnull(w['Conditions'].iloc[0])):
            con = w['Conditions'].iloc[0]
        if (not pd.isnull(w['TemperatureC'].iloc[0])):
            temp = w['TemperatureC'].iloc[0]
        if (not pd.isnull(w['Precipitationmm'].iloc[0])):
            rainmm = w['Precipitationmm'].iloc[0]
        if (not pd.isnull(w['VisibilityKm'].iloc[0])):
            viskm = w['VisibilityKm'].iloc[0]
            
        #print 'con %s, temp %s, rainmm %s, viskm %s' % (con, temp, rainmm, viskm)
        
        #print "2.75"
        s = pd.Series([con, rainmm, temp, viskm])
        #print "3"
        #print str(len(w.values[0]))
        #s = pd.Series(w.values[0])
        #s = pd.Series([w['Conditions'].iloc[0], w['Dew PointC'].iloc[0], w['Gust SpeedKm/h'].iloc[0]])

        #s.columns = features
        return s
    except:
        #print "4"
        print date + "x" + time
        s = pd.Series([None,None,None,None])
        #s = pd.Series(["1","2","3","4","5","6","7","8","9"])
        #s = pd.Series([])
        #s.columns = features
        return s
    
    
    

#lookup_weather2(2016, 2, 14, 7)
#lookup_weather('03/14/2016', '3:27').values[0]
#[unnamed, condition, dateUTC, Dew, Events, Gust, Humidity,Precipitationmm,Sea_Level_PressurehPa, TemperatureC] = lookup_weather('01/27/2016', '3:27').values[0]

In [147]:
print "Applying weather data to incidents..."
incidents[features] = incidents[incidents.DATE.str.split('/').str.get(2) != '2016'].apply(merge_weather, axis=1)
print "Saving weather in-riched incident data..."
incidents.to_csv('datasets/NYPD_Motor_Vehicle_Collisions_weather3.csv', sep=',')

Applying weather data to incidents...
07/26/2014x11:15
07/26/2014x11:30
07/26/2014x11:40
07/26/2014x11:45
07/26/2014x11:45
07/26/2014x11:45
07/26/2014x12:03
07/26/2014x12:11
07/26/2014x12:15
07/26/2014x12:15
07/26/2014x12:30
07/26/2014x12:30
07/26/2014x11:25
07/26/2014x11:25
07/26/2014x11:30
07/26/2014x11:30
07/26/2014x12:15
07/26/2014x12:16
07/26/2014x12:20
07/26/2014x12:30
07/26/2014x12:30
07/26/2014x11:00
07/26/2014x11:00
07/26/2014x11:55
07/26/2014x11:00
07/26/2014x11:05
07/26/2014x11:10
07/26/2014x11:10
07/26/2014x11:15
07/26/2014x11:15
07/26/2014x11:18
07/26/2014x11:47
07/26/2014x11:50
07/26/2014x11:50
07/26/2014x11:55
07/26/2014x12:00
07/26/2014x12:30
07/26/2014x12:35
07/26/2014x12:40
07/26/2014x12:45
07/26/2014x11:00
07/26/2014x11:00
07/26/2014x12:15
07/26/2014x12:48
05/06/2014x5:50
05/06/2014x5:40
05/06/2014x5:40
05/06/2014x5:00
05/06/2014x5:20
05/06/2014x5:44
05/06/2014x5:55
09/18/2012x2:20
09/18/2012x2:30
09/18/2012x3:20
09/18/2012x2:55
09/18/2012x2:22
09/18/2012x1:45
09/18/

In [11]:
incidents[incidents.DATE.str.split('/').str.get(2) == '2016']

,DATE,TIME,BOROUGH,ZIP CODE,LATITUDE,LONGITUDE,LOCATION,ON STREET NAME,CROSS STREET NAME,OFF STREET NAME,NUMBER OF PERSONS INJURED,NUMBER OF PERSONS KILLED,NUMBER OF PEDESTRIANS INJURED,NUMBER OF PEDESTRIANS KILLED,NUMBER OF CYCLIST INJURED,NUMBER OF CYCLIST KILLED,NUMBER OF MOTORIST INJURED,NUMBER OF MOTORIST KILLED,CONTRIBUTING FACTOR VEHICLE 1,CONTRIBUTING FACTOR VEHICLE 2,CONTRIBUTING FACTOR VEHICLE 3,CONTRIBUTING FACTOR VEHICLE 4,CONTRIBUTING FACTOR VEHICLE 5,UNIQUE KEY,VEHICLE TYPE CODE 1,VEHICLE TYPE CODE 2,VEHICLE TYPE CODE 3,VEHICLE TYPE CODE 4,VEHICLE TYPE CODE 5
0,03/14/2016,3:27,QUEENS,11372.0,40.747734,-73.882999,"(40.7477341, -73.8829986)",ROOSEVELT AVENUE,83 STREET,NaN,1,0,1,0,0,0,0,0,Unspecified,NaN,NaN,NaN,NaN,3405169,OTHER,NaN,NaN,NaN,NaN
1,03/14/2016,2:00,NaN,NaN,NaN,NaN,NaN,PULASKI BRIDGE,EAGLE STREET,NaN,0,0,0,0,0,0,0,0,Unspecified,Unspecified,NaN,NaN,NaN,3405134,PASSENGER VEHICLE,OTHER,NaN,NaN,NaN
2,03/14/2016,2:40,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0,Other Vehicular,Unspecified,NaN,NaN,NaN,3405070,PASSENGER VEHICLE,OTHER,NaN,NaN,NaN
3,03/14/2016,0:45,MANHATTAN,10035.0,40.808279,-73.938793,"(40.8082795, -73.9387929)",EAST 129 STREET,MADISON AVENUE,NaN,0,0,0,0,0,0,0,0,Unspecified,Unspecified,NaN,NaN,NaN,3405059,PASSENGER VEHICLE,PASSENGER VEHICLE,NaN,NaN,NaN
4,03/13/2016,23:00,BROOKLYN,11206.0,40.706653,-73.950406,"(40.7066527, -73.9504063)",UNION AVENUE,MONTROSE AVENUE,NaN,0,0,0,0,0,0,0,0,Driver Inattention/Distraction,Unspecified,NaN,NaN,NaN,3405121,PASSENGER VEHICLE,PASSENGER VEHICLE,NaN,NaN,NaN
5,03/13/2016,16:00,NaN,NaN,NaN,NaN,NaN,JAMAICA AVENUE,148 STREET,NaN,0,0,0,0,0,0,0,0,Failure to Yield Right-of-Way,Lost Consciousness,NaN,NaN,NaN,3404861,SPORT UTILITY / STATION WAGON,SPORT UTILITY / STATION WAGON,NaN,NaN,NaN
6,03/13/2016,9:48,BROOKLYN,11212.0,40.661997,-73.919593,"(40.661997, -73.9195931)",KINGS HIGHWAY,EAST 98 STREET,NaN,0,0,0,0,0,0,0,0,Passenger Distraction,Unspecified,NaN,NaN,NaN,3404744,PASSENGER VEHICLE,UNKNOWN,NaN,NaN,NaN
7,03/13/2016,14:25,NaN,NaN,NaN,NaN,NaN,BRADLEY AVENUE,BRIELLE AVENUE,NaN,0,0,0,0,0,0,0,0,Unspecified,Unspecified,NaN,NaN,NaN,3405034,SPORT UTILITY / STATION WAGON,SPORT UTILITY / STATION WAGON,NaN,NaN,NaN
8,03/13/2016,19:55,NaN,NaN,NaN,NaN,NaN,OCEANIA STREET,48 AVENUE,NaN,0,0,0,0,0,0,0,0,Failure to Yield Right-of-Way,NaN,NaN,NaN,NaN,3405164,SPORT UTILITY / STATION WAGON,NaN,NaN,NaN,NaN
9,03/13/2016,9:46,QUEENS,11106.0,40.756580,-73.929752,"(40.75658, -73.9297516)",36 AVENUE,31 STREET,NaN,0,0,0,0,0,0,0,0,Failure to Yield Right-of-Way,Unspecified,NaN,NaN,NaN,3404995,LIVERY VEHICLE,PASSENGER VEHICLE,NaN,NaN,NaN


## Make some nice data analysis

In [6]:
incidents = pd.read_csv('datasets/NYPD_Motor_Vehicle_Collisions.csv')
weather = pd.read_csv('datasets/weather_data_nyc_clean2.csv')

In [87]:
#03/02/2016x11:30

merge_weather(incidents[(incidents['DATE'] == '03/02/2016') & (incidents['TIME'] == '11:30')].iloc[3])

03/02/2016x11:30


0    None
1    None
2    None
3    None
4    None
5    None
6    None
7    None
8    None
dtype: object

In [47]:
incidents.tail(1).iloc[0]

DATE                                            07/01/2012
TIME                                                 14:40
BOROUGH                                      STATEN ISLAND
ZIP CODE                                             10303
LATITUDE                                           40.6368
LONGITUDE                                         -74.1579
LOCATION                         (40.6368184, -74.1579392)
ON STREET NAME                                UNION AVENUE
CROSS STREET NAME                         RICHMOND TERRACE
OFF STREET NAME                                        NaN
NUMBER OF PERSONS INJURED                                0
NUMBER OF PERSONS KILLED                                 0
NUMBER OF PEDESTRIANS INJURED                            0
NUMBER OF PEDESTRIANS KILLED                             0
NUMBER OF CYCLIST INJURED                                0
NUMBER OF CYCLIST KILLED                                 0
NUMBER OF MOTORIST INJURED                              

In [52]:
lookup_weather('07/01/2012', '14:40')

,Unnamed: 0,Conditions,DateUTC,Dew PointC,Events,Gust SpeedKm/h,Humidity,Precipitationmm,Sea Level PressurehPa,TemperatureC,TimeEDT,TimeEST,VisibilityKm,Wind Direction,Wind SpeedKm/h,WindDirDegrees,Year,Month,Day,Hour
14,14,Partly Cloudy,7/1/2012 18:51:00,18.3,NaN,NaN,39.0,NaN,1008.1,33.9,2:51 PM,NaN,16.1,WSW,11.1,240,2012,7,1,14


In [136]:
lookup_weather('03/02/2016', '11:30')

,Unnamed: 0,Conditions,DateUTC,Dew PointC,Events,Gust SpeedKm/h,Humidity,Precipitationmm,Sea Level PressurehPa,TemperatureC,TimeEDT,TimeEST,VisibilityKm,Wind Direction,Wind SpeedKm/h,WindDirDegrees,Year,Month,Day,Hour


In [139]:
dt = incidents[(incidents['DATE'] == '03/02/2016') & (incidents['TIME'] == '11:30')]
dt[features] = dt.apply(merge_weather, axis=1)

0
1
2
2.5
4
03/02/2016x11:30
0
1
2
2.5
4
03/02/2016x11:30
0
1
2
2.5
4
03/02/2016x11:30
0
1
2
2.5
4
03/02/2016x11:30
0
1
2
2.5
4
03/02/2016x11:30
